# More feature engineering with SQL (For predicting if a game is high scoring)

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# cnx = create_engine('postgresql://username:password@ip_address:port/database')
cnx = create_engine('postgresql://lucyallen@localhost:5432/nhl_new')

In [3]:
df = pd.read_sql_query('''select *, (hometenwinpct - awaytenwinpct) as tenwinpctdiff, (hometengoalsfor-awaytengoalsfor) as tengamegoalfordiff,
(homefivegoalsfor-awayfivegoalsfor) as fivegamegoalfordiff, (hometengoalsagainst-awaytengoalsagainst) as tengamegoalagainstdiff,
(homefivegoalsagainst-awayfivegoalsagainst) as fivegamegoalagainstdiff, (homefivegoalsfor-homefivegoalsagainst) as homefivegoaldiff, 
(hometengoalsfor-hometengoalsagainst) as hometengoaldiff, (awayfivegoalsfor-awayfivegoalsagainst) as awayfivegoaldiff,
(awaytengoalsfor-awaytengoalsagainst) as awaytengoaldiff,
case when homedivision = awaydivision then 1 else 0 end as samedivision,
case when homeresult = 'Win' then 1 else 0 end as win
from 
(select * 
from nhl_home_new nh 
left join nhl_away_new na 
on nh.hometeam = na.hometeam and na.date = nh.date
) as tablehome''', cnx)

###### Removing Duplicate Rows

In [4]:
nhl = df.loc[:,~df.columns.duplicated()]

In [5]:
nhl

,index,homedivision,hometeam,date,awayteam,homegoals,awaygoals,homeresult,datetime,totalpoints,...,tengamegoalfordiff,fivegamegoalfordiff,tengamegoalagainstdiff,fivegamegoalagainstdiff,homefivegoaldiff,hometengoaldiff,awayfivegoaldiff,awaytengoaldiff,samedivision,win
0,0,Atlantic,Tampa Bay,11/21/18,Florida,7,3,Win,2018-11-21,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
1,1,Atlantic,Tampa Bay,11/23/18,Chicago,4,2,Win,2018-11-23,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
2,2,Atlantic,Tampa Bay,11/25/18,New Jersey,5,2,Win,2018-11-25,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
3,3,Atlantic,Tampa Bay,11/27/18,Anaheim,1,3,Loss,2018-11-27,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,4,Atlantic,Tampa Bay,11/29/18,Buffalo,5,4,Win,2018-11-29,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,949,Pacific,Los Angeles,3/21/19,San Jose,4,2,Win,2019-03-21,6,...,-11.0,-4.0,4.0,-3.0,-5.0,-14.0,-4.0,1.0,1,1
950,950,Pacific,Los Angeles,3/23/19,Anaheim,4,3,Win,2019-03-23,7,...,-8.0,0.0,2.0,-3.0,-1.0,-7.0,-4.0,3.0,1,1
951,951,Pacific,Los Angeles,3/30/19,Chicago,3,2,Win,2019-03-30,5,...,0.0,7.0,10.0,3.0,1.0,-4.0,-3.0,6.0,0,1
952,952,Pacific,Los Angeles,4/1/19,Calgary,2,7,Loss,2019-04-01,9,...,-10.0,1.0,10.0,6.0,0.0,-1.0,5.0,19.0,1,0


In [8]:
nhl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954 entries, 0 to 953
Data columns (total 71 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    954 non-null    int64  
 1   homedivision             954 non-null    object 
 2   hometeam                 954 non-null    object 
 3   date                     954 non-null    object 
 4   awayteam                 954 non-null    object 
 5   homegoals                954 non-null    int64  
 6   awaygoals                954 non-null    int64  
 7   homeresult               954 non-null    object 
 8   datetime                 954 non-null    object 
 9   totalpoints              954 non-null    int64  
 10  homeprevpct              938 non-null    float64
 11  homeprevwins             938 non-null    float64
 12  homeprevloss             938 non-null    float64
 13  homeprevties             938 non-null    float64
 14  homefivepct              8

In [9]:
nhl

,index,homedivision,hometeam,date,awayteam,homegoals,awaygoals,homeresult,datetime,totalpoints,...,tengamegoalfordiff,fivegamegoalfordiff,tengamegoalagainstdiff,fivegamegoalagainstdiff,homefivegoaldiff,hometengoaldiff,awayfivegoaldiff,awaytengoaldiff,samedivision,win
0,0,Atlantic,Tampa Bay,11/21/18,Florida,7,3,Win,2018-11-21,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
1,1,Atlantic,Tampa Bay,11/23/18,Chicago,4,2,Win,2018-11-23,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
2,2,Atlantic,Tampa Bay,11/25/18,New Jersey,5,2,Win,2018-11-25,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
3,3,Atlantic,Tampa Bay,11/27/18,Anaheim,1,3,Loss,2018-11-27,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,4,Atlantic,Tampa Bay,11/29/18,Buffalo,5,4,Win,2018-11-29,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,949,Pacific,Los Angeles,3/21/19,San Jose,4,2,Win,2019-03-21,6,...,-11.0,-4.0,4.0,-3.0,-5.0,-14.0,-4.0,1.0,1,1
950,950,Pacific,Los Angeles,3/23/19,Anaheim,4,3,Win,2019-03-23,7,...,-8.0,0.0,2.0,-3.0,-1.0,-7.0,-4.0,3.0,1,1
951,951,Pacific,Los Angeles,3/30/19,Chicago,3,2,Win,2019-03-30,5,...,0.0,7.0,10.0,3.0,1.0,-4.0,-3.0,6.0,0,1
952,952,Pacific,Los Angeles,4/1/19,Calgary,2,7,Loss,2019-04-01,9,...,-10.0,1.0,10.0,6.0,0.0,-1.0,5.0,19.0,1,0


In [10]:
nhl.columns

Index(['index', 'homedivision', 'hometeam', 'date', 'awayteam', 'homegoals',
       'awaygoals', 'homeresult', 'datetime', 'totalpoints', 'homeprevpct',
       'homeprevwins', 'homeprevloss', 'homeprevties', 'homefivepct',
       'homefivewins', 'homefiveloss', 'homefiveties', 'hometenpct',
       'hometenwins', 'hometenloss', 'hometenties', 'homefivegoalsfor',
       'homefivegoalsagainst', 'hometengoalsfor', 'hometengoalsagainst',
       'homelastfivewins', 'homelastfiveloss', 'homelastfiveties',
       'homelasttenwins', 'homelasttenloss', 'homelasttenties',
       'homefivewinpct', 'hometenwinpct', 'awaydivision', 'awayresult',
       'awayprevpct', 'awayprevwins', 'awayprevloss', 'awayprevties',
       'awayfivepct', 'awayfivewins', 'awayfiveloss', 'awayfiveties',
       'awaytenpct', 'awaytenwins', 'awaytenloss', 'awaytenties',
       'awayfivegoalsfor', 'awayfivegoalsagainst', 'awaytengoalsfor',
       'awaytengoalsagainst', 'awaylastfivewins', 'awaylastfiveloss',
       'awayla

###### Dropping Null Values

In [12]:
nhl.dropna(inplace=True)

<ipython-input-12-727fdd3092e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhl.dropna(inplace=True)


In [13]:
nhl.reset_index(inplace=True)

In [14]:
nhl.columns

Index(['level_0', 'index', 'homedivision', 'hometeam', 'date', 'awayteam',
       'homegoals', 'awaygoals', 'homeresult', 'datetime', 'totalpoints',
       'homeprevpct', 'homeprevwins', 'homeprevloss', 'homeprevties',
       'homefivepct', 'homefivewins', 'homefiveloss', 'homefiveties',
       'hometenpct', 'hometenwins', 'hometenloss', 'hometenties',
       'homefivegoalsfor', 'homefivegoalsagainst', 'hometengoalsfor',
       'hometengoalsagainst', 'homelastfivewins', 'homelastfiveloss',
       'homelastfiveties', 'homelasttenwins', 'homelasttenloss',
       'homelasttenties', 'homefivewinpct', 'hometenwinpct', 'awaydivision',
       'awayresult', 'awayprevpct', 'awayprevwins', 'awayprevloss',
       'awayprevties', 'awayfivepct', 'awayfivewins', 'awayfiveloss',
       'awayfiveties', 'awaytenpct', 'awaytenwins', 'awaytenloss',
       'awaytenties', 'awayfivegoalsfor', 'awayfivegoalsagainst',
       'awaytengoalsfor', 'awaytengoalsagainst', 'awaylastfivewins',
       'awaylastfivelos

###### Dropping some unnecessary columns

In [15]:
nhl.drop(['level_0', 'index', 'homegoals', 'awaygoals', 'homefivepct', 'homefivewins', 'homefiveloss', 'homefiveties', 
         'hometenpct', 'hometenwins', 'hometenloss', 'hometenties', 'awayfivepct', 'awayfivewins', 'awayfiveloss', 'awayfiveties', 
         'awaytenpct', 'awaytenwins', 'awaytenloss', 'awaytenties'], axis=1, inplace=True)

/Users/lucyallen/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [16]:
nhl.describe()

,totalpoints,homeprevpct,homeprevwins,homeprevloss,homeprevties,homefivegoalsfor,homefivegoalsagainst,hometengoalsfor,hometengoalsagainst,homelastfivewins,...,tengamegoalfordiff,fivegamegoalfordiff,tengamegoalagainstdiff,fivegamegoalagainstdiff,homefivegoaldiff,hometengoaldiff,awayfivegoaldiff,awaytengoaldiff,samedivision,win
count,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,...,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000
mean,5.920103,0.498760,28.131443,22.117268,6.194588,14.722938,14.921392,29.798969,29.864691,2.463918,...,-0.166237,-0.315722,-0.094072,0.034794,-0.198454,-0.065722,0.152062,0.006443,0.362113,0.521907
std,2.310997,0.088479,8.985766,7.409003,2.484433,4.123799,4.203563,6.216590,6.323208,1.245891,...,8.902305,5.706550,8.920012,6.021902,6.207325,9.314273,6.238285,9.380623,0.480921,0.499842
min,1.000000,0.303000,10.000000,7.000000,1.000000,4.000000,3.000000,12.000000,13.000000,0.000000,...,-29.000000,-20.000000,-28.000000,-18.000000,-22.000000,-30.000000,-22.000000,-28.000000,0.000000,0.000000
25%,4.000000,0.425000,22.000000,16.000000,4.000000,12.000000,12.000000,26.000000,26.000000,2.000000,...,-6.000000,-4.000000,-6.000000,-4.000000,-5.000000,-6.000000,-4.000000,-6.000000,0.000000,0.000000
50%,6.000000,0.500000,27.000000,21.000000,6.000000,15.000000,15.000000,29.000000,30.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,7.000000,0.566000,34.000000,27.000000,8.000000,17.000000,17.000000,34.000000,34.000000,3.000000,...,6.000000,3.000000,6.000000,4.000000,4.000000,6.000000,5.000000,6.000000,1.000000,1.000000
max,15.000000,0.730000,59.000000,46.000000,14.000000,28.000000,30.000000,52.000000,47.000000,5.000000,...,23.000000,17.000000,25.000000,22.000000,20.000000,26.000000,19.000000,27.000000,1.000000,1.000000


In [17]:
nhl.drop(['homeprevpct', 'homeprevwins', 'homeprevloss', 'homeprevties'], axis=1, inplace=True)

/Users/lucyallen/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [18]:
nhl

,homedivision,hometeam,date,awayteam,homeresult,datetime,totalpoints,homefivegoalsfor,homefivegoalsagainst,hometengoalsfor,...,tengamegoalfordiff,fivegamegoalfordiff,tengamegoalagainstdiff,fivegamegoalagainstdiff,homefivegoaldiff,hometengoaldiff,awayfivegoaldiff,awaytengoaldiff,samedivision,win
0,Atlantic,Tampa Bay,12/27/18,Philadelphia,Win,2018-12-27,11,24.0,15.0,51.0,...,23.0,12.0,-10.0,1.0,9.0,24.0,-2.0,-9.0,0,1
1,Atlantic,Tampa Bay,12/29/18,Montreal,Win,2018-12-29,11,26.0,19.0,52.0,...,20.0,14.0,1.0,6.0,7.0,21.0,-1.0,2.0,1,1
2,Atlantic,Tampa Bay,1/8/19,Columbus,Win,2019-01-08,4,22.0,18.0,46.0,...,17.0,4.0,10.0,1.0,4.0,13.0,1.0,6.0,0,1
3,Atlantic,Tampa Bay,1/10/19,Carolina,Win,2019-01-10,4,20.0,13.0,46.0,...,18.0,-1.0,4.0,0.0,7.0,14.0,8.0,0.0,0,1
4,Atlantic,Tampa Bay,1/17/19,Toronto,Loss,2019-01-17,6,15.0,9.0,37.0,...,6.0,1.0,-5.0,-6.0,6.0,10.0,-1.0,-1.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,Pacific,Los Angeles,3/21/19,San Jose,Win,2019-03-21,6,11.0,16.0,22.0,...,-11.0,-4.0,4.0,-3.0,-5.0,-14.0,-4.0,1.0,1,1
772,Pacific,Los Angeles,3/23/19,Anaheim,Win,2019-03-23,7,13.0,14.0,25.0,...,-8.0,0.0,2.0,-3.0,-1.0,-7.0,-4.0,3.0,1,1
773,Pacific,Los Angeles,3/30/19,Chicago,Win,2019-03-30,5,17.0,16.0,28.0,...,0.0,7.0,10.0,3.0,1.0,-4.0,-3.0,6.0,0,1
774,Pacific,Los Angeles,4/1/19,Calgary,Loss,2019-04-01,9,16.0,16.0,29.0,...,-10.0,1.0,10.0,6.0,0.0,-1.0,5.0,19.0,1,0


In [19]:
nhl.drop(['awayprevpct', 'awayprevwins', 'awayprevloss', 'awayprevties'], axis=1, inplace=True)

/Users/lucyallen/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [20]:
nhl.columns

Index(['homedivision', 'hometeam', 'date', 'awayteam', 'homeresult',
       'datetime', 'totalpoints', 'homefivegoalsfor', 'homefivegoalsagainst',
       'hometengoalsfor', 'hometengoalsagainst', 'homelastfivewins',
       'homelastfiveloss', 'homelastfiveties', 'homelasttenwins',
       'homelasttenloss', 'homelasttenties', 'homefivewinpct', 'hometenwinpct',
       'awaydivision', 'awayresult', 'awayfivegoalsfor',
       'awayfivegoalsagainst', 'awaytengoalsfor', 'awaytengoalsagainst',
       'awaylastfivewins', 'awaylastfiveloss', 'awaylastfiveties',
       'awaylasttenwins', 'awaylasttenloss', 'awaylasttenties',
       'awayfivewinpct', 'awaytenwinpct', 'tenwinpctdiff',
       'tengamegoalfordiff', 'fivegamegoalfordiff', 'tengamegoalagainstdiff',
       'fivegamegoalagainstdiff', 'homefivegoaldiff', 'hometengoaldiff',
       'awayfivegoaldiff', 'awaytengoaldiff', 'samedivision', 'win'],
      dtype='object')

###### Creating some new columns for total goals in the games since that will probably be more important with the high goal game prediction

In [21]:
nhl['homefivetotalgoals'] = nhl['homefivegoalsfor'] + nhl['homefivegoalsagainst']

<ipython-input-21-97964718779b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhl['homefivetotalgoals'] = nhl['homefivegoalsfor'] + nhl['homefivegoalsagainst']


In [22]:
nhl['hometentotalgoals'] = nhl['hometengoalsfor'] + nhl['hometengoalsagainst']

<ipython-input-22-59baf65a1e3d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhl['hometentotalgoals'] = nhl['hometengoalsfor'] + nhl['hometengoalsagainst']


In [23]:
nhl['awayfivetotalgoals'] = nhl['awayfivegoalsfor'] + nhl['awayfivegoalsagainst']

<ipython-input-23-c20beb4a6193>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhl['awayfivetotalgoals'] = nhl['awayfivegoalsfor'] + nhl['awayfivegoalsagainst']


In [24]:
nhl['awaytentotalgoals'] = nhl['awaytengoalsfor'] + nhl['awaytengoalsagainst']

<ipython-input-24-ad24db3d7ba7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhl['awaytentotalgoals'] = nhl['awaytengoalsfor'] + nhl['awaytengoalsagainst']


In [25]:
nhl.columns

Index(['homedivision', 'hometeam', 'date', 'awayteam', 'homeresult',
       'datetime', 'totalpoints', 'homefivegoalsfor', 'homefivegoalsagainst',
       'hometengoalsfor', 'hometengoalsagainst', 'homelastfivewins',
       'homelastfiveloss', 'homelastfiveties', 'homelasttenwins',
       'homelasttenloss', 'homelasttenties', 'homefivewinpct', 'hometenwinpct',
       'awaydivision', 'awayresult', 'awayfivegoalsfor',
       'awayfivegoalsagainst', 'awaytengoalsfor', 'awaytengoalsagainst',
       'awaylastfivewins', 'awaylastfiveloss', 'awaylastfiveties',
       'awaylasttenwins', 'awaylasttenloss', 'awaylasttenties',
       'awayfivewinpct', 'awaytenwinpct', 'tenwinpctdiff',
       'tengamegoalfordiff', 'fivegamegoalfordiff', 'tengamegoalagainstdiff',
       'fivegamegoalagainstdiff', 'homefivegoaldiff', 'hometengoaldiff',
       'awayfivegoaldiff', 'awaytengoaldiff', 'samedivision', 'win',
       'homefivetotalgoals', 'hometentotalgoals', 'awayfivetotalgoals',
       'awaytentotalgoals'

In [27]:
nhl

,homedivision,hometeam,date,awayteam,homeresult,datetime,totalpoints,homefivegoalsfor,homefivegoalsagainst,hometengoalsfor,...,hometengoaldiff,awayfivegoaldiff,awaytengoaldiff,samedivision,win,homefivetotalgoals,hometentotalgoals,awayfivetotalgoals,awaytentotalgoals,over
0,Atlantic,Tampa Bay,12/27/18,Philadelphia,Win,2018-12-27,11,24.0,15.0,51.0,...,24.0,-2.0,-9.0,0,1,39.0,78.0,26.0,65.0,0
1,Atlantic,Tampa Bay,12/29/18,Montreal,Win,2018-12-29,11,26.0,19.0,52.0,...,21.0,-1.0,2.0,1,1,45.0,83.0,25.0,62.0,0
2,Atlantic,Tampa Bay,1/8/19,Columbus,Win,2019-01-08,4,22.0,18.0,46.0,...,13.0,1.0,6.0,0,1,40.0,79.0,35.0,52.0,0
3,Atlantic,Tampa Bay,1/10/19,Carolina,Win,2019-01-10,4,20.0,13.0,46.0,...,14.0,8.0,0.0,0,1,33.0,78.0,34.0,56.0,0
4,Atlantic,Tampa Bay,1/17/19,Toronto,Loss,2019-01-17,6,15.0,9.0,37.0,...,10.0,-1.0,-1.0,1,0,24.0,64.0,29.0,63.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,Pacific,Los Angeles,3/21/19,San Jose,Win,2019-03-21,6,11.0,16.0,22.0,...,-14.0,-4.0,1.0,1,1,27.0,58.0,34.0,65.0,0
772,Pacific,Los Angeles,3/23/19,Anaheim,Win,2019-03-23,7,13.0,14.0,25.0,...,-7.0,-4.0,3.0,1,1,27.0,57.0,30.0,63.0,0
773,Pacific,Los Angeles,3/30/19,Chicago,Win,2019-03-30,5,17.0,16.0,28.0,...,-4.0,-3.0,6.0,0,1,33.0,60.0,23.0,50.0,0
774,Pacific,Los Angeles,4/1/19,Calgary,Loss,2019-04-01,9,16.0,16.0,29.0,...,-1.0,5.0,19.0,1,0,32.0,59.0,25.0,59.0,0


###### Creating a target variable `over` to use, using > 7 total points since it is the 75th percentile

In [28]:
nhl['over'] = 0
for i in range(776):
    if nhl.loc[i, 'totalpoints'] > 7:
        nhl.loc[i,'over'] = 1

<ipython-input-28-7e796e3d0d6d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhl['over'] = 0
/Users/lucyallen/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [29]:
nhl

,homedivision,hometeam,date,awayteam,homeresult,datetime,totalpoints,homefivegoalsfor,homefivegoalsagainst,hometengoalsfor,...,hometengoaldiff,awayfivegoaldiff,awaytengoaldiff,samedivision,win,homefivetotalgoals,hometentotalgoals,awayfivetotalgoals,awaytentotalgoals,over
0,Atlantic,Tampa Bay,12/27/18,Philadelphia,Win,2018-12-27,11,24.0,15.0,51.0,...,24.0,-2.0,-9.0,0,1,39.0,78.0,26.0,65.0,1
1,Atlantic,Tampa Bay,12/29/18,Montreal,Win,2018-12-29,11,26.0,19.0,52.0,...,21.0,-1.0,2.0,1,1,45.0,83.0,25.0,62.0,1
2,Atlantic,Tampa Bay,1/8/19,Columbus,Win,2019-01-08,4,22.0,18.0,46.0,...,13.0,1.0,6.0,0,1,40.0,79.0,35.0,52.0,0
3,Atlantic,Tampa Bay,1/10/19,Carolina,Win,2019-01-10,4,20.0,13.0,46.0,...,14.0,8.0,0.0,0,1,33.0,78.0,34.0,56.0,0
4,Atlantic,Tampa Bay,1/17/19,Toronto,Loss,2019-01-17,6,15.0,9.0,37.0,...,10.0,-1.0,-1.0,1,0,24.0,64.0,29.0,63.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,Pacific,Los Angeles,3/21/19,San Jose,Win,2019-03-21,6,11.0,16.0,22.0,...,-14.0,-4.0,1.0,1,1,27.0,58.0,34.0,65.0,0
772,Pacific,Los Angeles,3/23/19,Anaheim,Win,2019-03-23,7,13.0,14.0,25.0,...,-7.0,-4.0,3.0,1,1,27.0,57.0,30.0,63.0,0
773,Pacific,Los Angeles,3/30/19,Chicago,Win,2019-03-30,5,17.0,16.0,28.0,...,-4.0,-3.0,6.0,0,1,33.0,60.0,23.0,50.0,0
774,Pacific,Los Angeles,4/1/19,Calgary,Loss,2019-04-01,9,16.0,16.0,29.0,...,-1.0,5.0,19.0,1,0,32.0,59.0,25.0,59.0,1


### Saving this table to import into a new notebook for modeling

In [30]:
nhl.to_csv('nhl_over_under_table.csv')

In [31]:
nhl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 49 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   homedivision             776 non-null    object 
 1   hometeam                 776 non-null    object 
 2   date                     776 non-null    object 
 3   awayteam                 776 non-null    object 
 4   homeresult               776 non-null    object 
 5   datetime                 776 non-null    object 
 6   totalpoints              776 non-null    int64  
 7   homefivegoalsfor         776 non-null    float64
 8   homefivegoalsagainst     776 non-null    float64
 9   hometengoalsfor          776 non-null    float64
 10  hometengoalsagainst      776 non-null    float64
 11  homelastfivewins         776 non-null    float64
 12  homelastfiveloss         776 non-null    float64
 13  homelastfiveties         776 non-null    float64
 14  homelasttenwins          7